In [3]:
import collections

import tensorflow as tf
import tensorflow_federated as tff

# Implementing Federated Averaging

In [6]:
NUMBER_FEATURES = 784
NUMBER_TARGETS = 10

## Combining TensorFlow and TensorFlow Federated

# Defining a loss function

In [4]:
BATCH_SPEC = collections.OrderedDict(
    X=tf.TensorSpec(shape=[None, NUMBER_FEATURES], dtype=tf.float32),
    y=tf.TensorSpec(shape=[None], dtype=tf.int32)
)
BATCH_TYPE = tff.to_type(BATCH_SPEC)

In [5]:
print(BATCH_TYPE)

<X=float32[?,784],y=int32[?]>


In [7]:
MODEL_SPEC = collections.OrderedDict(
    weights=tf.TensorSpec(shape=[NUMBER_FEATURES, NUMBER_TARGETS], dtype=tf.float32),
    bias=tf.TensorSpec(shape=[NUMBER_TARGETS], dtype=tf.float32)
)
MODEL_TYPE = tff.to_type(MODEL_SPEC)

In [8]:
print(MODEL_TYPE)

<weights=float32[784,10],bias=float32[10]>


In [33]:
@tf.function
def linear_transformation(model, batch):
    W = model["weights"]
    b = model["bias"]
    X = batch['X']
    Z = tf.matmul(X, W) + b
    return Z


@tf.function
def crossentropy_loss(y_true, y_pred):
    crossentropy = tf.reduce_sum(tf.one_hot(y_true, NUMBER_TARGETS) * tf.math.log(y_pred),  axis=[1])
    return -tf.reduce_mean(crossentropy)


@tff.tf_computation(MODEL_TYPE, BATCH_TYPE)
@tf.function
def batch_loss(model, batch):
    y_true = batch['y']
    y_pred = tf.nn.softmax(linear_transformation(model, batch))
    return crossentropy_loss(y_true, y_pred)

In [34]:
print(batch_loss.type_signature)

(<<weights=float32[784,10],bias=float32[10]>,<X=float32[?,784],y=int32[?]>> -> float32)


## Gradient descent on a single batch

In [31]:
@tff.tf_computation(MODEL_TYPE, BATCH_TYPE, tf.float32)
def batch_train(initial_model, batch, learning_rate):
    # Define a group of model variables and set them to `initial_model`. Must
    # be defined outside the @tf.function.
    model_vars = collections.OrderedDict([
        (name, tf.Variable(name=name, initial_value=value))
        for name, value in initial_model.items()
    ])
    optimizer = tf.keras.optimizers.SGD(learning_rate)

    @tf.function
    def _train_on_batch(model_vars, batch):
        # Perform one step of gradient descent using loss from `batch_loss`.
        with tf.GradientTape() as tape:
            loss = forward_pass(model_vars, batch)
        grads = tape.gradient(loss, model_vars)
        optimizer.apply_gradients(zip(tf.nest.flatten(grads), tf.nest.flatten(model_vars)))
        return model_vars

    return _train_on_batch(model_vars, batch)


NameError: in converted code:

    <ipython-input-24-fcb9a7e73b84>:15 _train_on_batch  *
        loss = forward_pass(model_vars, batch)
    /Users/pughdr/Training/data-science-project-templates/tensorflow-federated-data-science-project/env/lib/python3.7/site-packages/tensorflow_federated/python/core/impl/utils/function_utils.py:679 __call__  *
        arg = pack_args(self._type_signature.parameter, args, kwargs, context)
    /Users/pughdr/Training/data-science-project-templates/tensorflow-federated-data-science-project/env/lib/python3.7/site-packages/tensorflow_federated/python/core/impl/utils/function_utils.py:429 pack_args  *
        elif not is_argument_tuple(parameter_type):
    /Users/pughdr/Training/data-science-project-templates/tensorflow-federated-data-science-project/env/lib/python3.7/site-packages/tensorflow_federated/python/core/impl/utils/function_utils.py:258 is_argument_tuple  *
        for idx, element in enumerate(elements):
    /Users/pughdr/Training/data-science-proj